In [22]:
import pandas as pd
import plotly.express as px
import glob

import re
import os


In [57]:
files = glob.glob('../lulc_refine_outputs/*.txt')

name_map = {
    'boundary_naip_ndvi_surface' : 'w/ NDVI + Surface',
    'boundary_naip_ndvi' : 'w/ NDVI',
    'boundary_naip_nir' : 'w/ NIR',
    'nll_unconstrained' :'NAIP only (NLL)',
    'boundary_naip_surface' : 'w/ Surface',
    'weighted_nll_unconstrained' : 'NAIP only (weighted NLL)',
    'boundary_unconstrained' : 'NAIP only',
    'dynamicworld_baseline' : 'DynamicWorld (No Refinement)',
    'boundary_naip_nir_ndvi_surface' : 'w/ NIR + NDVI + Surface',
    'boundary_naip_nir_surface' : 'w/ NIR + Surface',
 }

In [58]:
def parse_file(filepath):
    exp_name = os.path.basename(filepath).split('.')[0]
    data = {'name' : name_map[exp_name]}

    data['objective'] = 'None'
    if 'nll' in exp_name:
        data['objective'] = 'NLL'
    if 'boundary' in exp_name:
        data['objective'] = 'Boundary'

    with open(filepath, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if 'mIoU' in line:
                data['mIoU'] = float(re.sub(r"([^0-9-<.]+)","", line.split(' ')[-1]))
            if 'Boundary' in line:
                data['Boundary F1'] = float(re.sub(r"([^0-9-<.]+)","", line.split(' ')[-1]))           
            if 'water' in line:
                data['Water'] = float(line.split(' ')[-1])
            if 'trees' in line:
                data['Trees'] = float(line.split(' ')[-1])
            if 'low vegetation' in line:
                data['Low Veg.'] = float(line.split(' ')[-1])
            if 'scrub and shrub' in line:
                data['Shrub and Scrub'] = float(line.split(' ')[-1])
            if 'flooded vegetation' in line:
                data['Flooded Veg.'] = float(line.split(' ')[-1])
            if 'built' in line:
                data['Built'] = float(line.split(' ')[-1])
            if 'bare' in line:
                data['Bare'] = float(line.split(' ')[-1])
    return data



In [59]:
df = pd.DataFrame()
for filepath in files:
    row = parse_file(filepath)
    df_row = pd.DataFrame([row])
    df = pd.concat([df_row, df], ignore_index=True)

In [60]:
df2 = df[['name', 'objective', 'Boundary F1', 'mIoU', 'Water', 'Trees', 'Low Veg.', 'Shrub and Scrub', 'Flooded Veg.', 'Built', 'Bare']]

In [61]:
df2.sort_values(by=['objective', 'Boundary F1', 'mIoU'], ascending=False, inplace=True)

In [62]:
df2

,name,objective,Boundary F1,mIoU,Water,Trees,Low Veg.,Shrub and Scrub,Flooded Veg.,Built,Bare
7,DynamicWorld (No Refinement),None,0.9450,0.431761,0.935045,0.648835,0.512701,0.002115,0.082492,0.234516,0.606624
5,NAIP only (weighted NLL),NLL,0.9144,0.441496,0.933532,0.668916,0.529757,0.001830,0.061785,0.251842,0.642810
3,NAIP only (NLL),NLL,0.9138,0.441619,0.933677,0.669150,0.529668,0.001832,0.061932,0.252001,0.643075
4,w/ Surface,Boundary,0.7842,0.447607,0.940562,0.678930,0.535426,0.001823,0.070431,0.265919,0.640155
6,NAIP only,Boundary,0.7766,0.452105,0.943709,0.693069,0.533331,0.001743,0.056738,0.281779,0.654369
1,w/ NDVI,Boundary,0.7749,0.446575,0.940166,0.686274,0.532959,0.001261,0.038295,0.285416,0.641653
0,w/ NDVI + Surface,Boundary,0.7682,0.452686,0.942848,0.703894,0.564624,0.001007,0.061408,0.269373,0.625652
2,w/ NIR,Boundary,0.7488,0.452674,0.952990,0.688299,0.535491,0.001291,0.049527,0.292860,0.648260
9,w/ NIR + Surface,Boundary,0.7399,0.456481,0.955465,0.695081,0.555947,0.001428,0.051110,0.286644,0.649692
8,w/ NIR + NDVI + Surface,Boundary,0.7375,0.456073,0.957578,0.690709,0.533720,0.000667,0.047788,0.315802,0.646245


In [63]:
print(df2.to_latex(index=False, float_format="%.3f", multirow=True))

\begin{tabular}{llrrrrrrrrr}
\toprule
name & objective & Boundary F1 & mIoU & Water & Trees & Low Veg. & Shrub and Scrub & Flooded Veg. & Built & Bare \\
\midrule
DynamicWorld (No Refinement) & None & 0.945 & 0.432 & 0.935 & 0.649 & 0.513 & 0.002 & 0.082 & 0.235 & 0.607 \\
NAIP only (weighted NLL) & NLL & 0.914 & 0.441 & 0.934 & 0.669 & 0.530 & 0.002 & 0.062 & 0.252 & 0.643 \\
NAIP only (NLL) & NLL & 0.914 & 0.442 & 0.934 & 0.669 & 0.530 & 0.002 & 0.062 & 0.252 & 0.643 \\
w/ Surface & Boundary & 0.784 & 0.448 & 0.941 & 0.679 & 0.535 & 0.002 & 0.070 & 0.266 & 0.640 \\
NAIP only & Boundary & 0.777 & 0.452 & 0.944 & 0.693 & 0.533 & 0.002 & 0.057 & 0.282 & 0.654 \\
w/ NDVI & Boundary & 0.775 & 0.447 & 0.940 & 0.686 & 0.533 & 0.001 & 0.038 & 0.285 & 0.642 \\
w/ NDVI + Surface & Boundary & 0.768 & 0.453 & 0.943 & 0.704 & 0.565 & 0.001 & 0.061 & 0.269 & 0.626 \\
w/ NIR & Boundary & 0.749 & 0.453 & 0.953 & 0.688 & 0.535 & 0.001 & 0.050 & 0.293 & 0.648 \\
w/ NIR + Surface & Boundary & 0.740 & 0